In [ ]:
# OVERLEAF
# https://www.overleaf.com/2783679526vchrbtrxmycm#bc3f9d

In [ ]:
# !pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from skopt import BayesSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Wczytanie wybranych zbiroków
christine = fetch_openml(data_id = 41142)
kddc = fetch_openml(data_id = 44035)
click = fetch_openml(data_id = 1220)
santa_nder = fetch_openml(data_id = 45566)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning

In [ ]:
# Sprawdzenie liczby wierszy
print(len(christine.data))
print(len(kddc.data))
print(len(click.data))
print(len(santa_nder.data))

5418
5032
39948
200000


In [ ]:
# Wylosowanie wierszy, które będą używane do obliczeń w przypadku zbioru santa_nder
num_samples = len(santa_nder.data)
random_indices = np.random.choice(num_samples, size=10000, replace=False)

In [ ]:
# Wczytanie zmiennych objasńianych i objaśniających
christine_X = christine.data
christine_y = christine.target
kddc_X = kddc.data
kddc_y = kddc.target
click_X = click.data
click_y = click.target
santa_nder_X = santa_nder.data.iloc[random_indices]
santa_nder_y = santa_nder.target.iloc[random_indices]

In [ ]:
# Sprawdzenie typu kolumn zbiorków
print(christine_X.dtypes)
print(kddc_X.dtypes)
print(click_X.dtypes)
print(santa_nder_X.dtypes)

V1       float64
V2       float64
V3       float64
V4       float64
V5       float64
          ...   
V1632    float64
V1633    float64
V1634    float64
V1635    float64
V1636    float64
Length: 1636, dtype: object
Var6       float64
Var13      float64
Var21      float64
Var22      float64
Var24      float64
Var25      float64
Var28      float64
Var35      float64
Var38      float64
Var57      float64
Var65      float64
Var73      float64
Var74      float64
Var76      float64
Var78      float64
Var81      float64
Var83      float64
Var85      float64
Var109     float64
Var112     float64
Var113     float64
Var119     float64
Var123     float64
Var125     float64
Var126     float64
Var132     float64
Var133     float64
Var134     float64
Var140     float64
Var144     float64
Var149     float64
Var153     float64
Var160     float64
Var163     float64
Var196    category
Var203    category
Var205    category
Var207    category
Var208    category
Var210    category
Var211    category
Var218

In [ ]:
# Sprawdzenie kolumn typu kategorycznego w przypadku zbioru christine
categorical_columns_christine = christine_X.select_dtypes(include=['object', 'category']).columns
categorical_columns_christine

Index(['V73', 'V156', 'V214', 'V239', 'V245', 'V257', 'V262', 'V478', 'V496',
       'V546', 'V566', 'V575', 'V584', 'V588', 'V647', 'V685', 'V688', 'V741',
       'V780', 'V833', 'V857', 'V935', 'V951', 'V1005', 'V1010', 'V1048',
       'V1187', 'V1229', 'V1239', 'V1240', 'V1256', 'V1260', 'V1301', 'V1356',
       'V1487', 'V1586', 'V1607'],
      dtype='object')

In [ ]:
df = pd.DataFrame(kddc_X)

# Oddzielenie kolumn typu 'number' od 'category'
numeric_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(include=['category']).columns

# One-hot encode kolumn kategorycznych
kddc_X_transform = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Sprawdzenie zmiennionej macierzy
print(kddc_X_transform)

        Var6   Var13   Var21   Var22  Var24  Var25   Var28  Var35      Var38  \
0     9723.0   908.0  4568.0  5710.0   54.0  840.0  287.76    0.0  3712122.0   
1      840.0  3108.0   212.0   265.0    6.0   88.0  166.56    0.0    15900.0   
2      259.0     0.0   128.0   160.0    6.0  128.0  200.00    0.0        0.0   
3      630.0   596.0    24.0    30.0    0.0    0.0  186.64    0.0  5485038.0   
4      140.0     0.0    64.0    80.0    0.0   24.0  220.08    0.0  4279404.0   
...      ...     ...     ...     ...    ...    ...     ...    ...        ...   
5027  1729.0  5024.0   176.0   220.0    2.0   32.0  220.08    0.0  9546480.0   
5028   749.0   196.0   148.0   185.0    6.0  128.0  166.56    0.0        0.0   
5029   511.0   104.0   116.0   145.0    4.0   96.0  167.20    0.0   212952.0   
5030  1295.0  1672.0   420.0   525.0    6.0  272.0  210.00   10.0  2131206.0   
5031  1694.0   828.0   192.0   240.0    0.0   48.0  220.08    5.0  2691228.0   

         Var57  ...  Var223_1  Var223_2

### Część I - Lasy losowe


In [ ]:
# Określnie zakresu hiperparametrów dla lasów losowych
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 2000, num = 20)]
max_depth = [int(x) for x in np.linspace(1, 30, num = 15)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 60, num = 20)]
min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 60, num = 20)]
max_samples = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
ccp_alpha = [float(x) for x in np.linspace(start = 0, stop = 1, num = 15)]

# Tworzennie siatki hiperparametrów
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_samples':max_samples,
               'ccp_alpha':ccp_alpha}

print(random_grid)

{'n_estimators': [1, 106, 211, 316, 421, 527, 632, 737, 842, 947, 1053, 1158, 1263, 1368, 1473, 1579, 1684, 1789, 1894, 2000], 'max_depth': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 30, None], 'min_samples_split': [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 60], 'min_samples_leaf': [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 32, 35, 38, 41, 44, 47, 50, 53, 56, 60], 'max_samples': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'ccp_alpha': [0.0, 0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.2857142857142857, 0.3571428571428571, 0.42857142857142855, 0.5, 0.5714285714285714, 0.6428571428571428, 0.7142857142857142, 0.7857142857142857, 0.8571428571428571, 0.9285714285714285, 1.0]}


Obliczenia dla Random Search w przypadku lasów losowych

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(click_X, click_y)
score_click = rf_random.cv_results_

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(christine_X, christine_y)
score_christine = rf_random.cv_results_

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(santa_nder_X, santa_nder_y)
score_santa_nder = rf_random.cv_results_

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter =50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(kddc_X_transform, kddc_y)
score_kddc = rf_random.cv_results_

In [ ]:
# Zapisywanie wyników
np.save('randomforestclassifier_score_click.npy', score_click)
np.save('randomforestclassifier_score_christine.npy', score_christine)
np.save('randomforestclassifier_score_santa_nder.npy', score_santa_nder)
np.save('randomforestclassifier_score_kddc.npy', score_kddc)

Obliczenia metodą Bayesowską dla lasów losowych

In [ ]:
clf = RandomForestClassifier()
opt = BayesSearchCV(clf, random_grid, n_iter=30, cv=3, random_state=42)
_ = opt.fit(christine_X, christine_y)
ob_rf_results_christine = opt.cv_results_

In [ ]:
clf = RandomForestClassifier()
opt = BayesSearchCV(clf, random_grid, n_iter=50, random_state=42)
_ = opt.fit(click_X, click_y)
ob_rf_results_click = opt.cv_results_


In [ ]:
clf = RandomForestClassifier()
opt = BayesSearchCV(clf, random_grid, n_iter=30, cv=3, random_state=42)
_ = opt.fit(santa_nder_X, santa_nder_y)
ob_rf_results_santa_nder = opt.cv_results_

In [ ]:
clf = RandomForestClassifier()
opt = BayesSearchCV(clf, random_grid, n_iter=50, random_state=42)
_ = opt.fit(kddc_X_transform, kddc_y)
ob_rf_results_kddc = opt.cv_results_

In [ ]:
# Zapisywanie wyników
np.save('op_rf_score_click.npy', ob_rf_results_click)
np.save('op_rf_score_christine.npy', ob_rf_results_christine)
np.save('op_rf_score_santa_nder.npy', ob_rf_results_santa_nder)
np.save('op_rf_score_kddc.npy', ob_rf_results_kddc)

### Część II - KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Określnie zakresu hiperparametrów dla lasów losowych i stworzenie siatki
param_grid = {
    'n_neighbors': [int(x) for x in np.linspace(start = 1, stop = 30, num = 30)],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 - odległość Manhattan'a, 2 - odległość euklidesowa
}

Obliczenia dla Random Search w przypadku KNN

In [ ]:
knn = KNeighborsClassifier()
rand_search = RandomizedSearchCV(estimator = knn, param_distributions = param_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rand_search.fit(click_X, click_y)
score_knn_click = rand_search.cv_results_

In [ ]:
knn = KNeighborsClassifier()
rand_search = RandomizedSearchCV(estimator = knn, param_distributions = param_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rand_search.fit(christine_X, christine_y)
score_knn_christine = rand_search.cv_results_

In [ ]:
knn = KNeighborsClassifier()
rand_search = RandomizedSearchCV(estimator = knn, param_distributions = param_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rand_search.fit(santa_nder_X, santa_nder_y)
score_knn_santa_nder = rand_search.cv_results_

In [ ]:
knn = KNeighborsClassifier()
rand_search = RandomizedSearchCV(estimator = knn, param_distributions = param_grid, n_iter = 50, cv = 7, verbose=2, random_state=42, n_jobs = -1)
rand_search.fit(kddc_X_transform, kddc_y)
score_knn_kddc = rand_search.cv_results_

In [ ]:
# Zapisywanie wyników
np.save('knn_score_click.npy', score_knn_click)
np.save('knn_score_christine.npy', score_knn_christine)
np.save('knn_score_santa_nder.npy', score_knn_santa_nder)
np.save('knn_score_kddc.npy', score_knn_kddc)

Obliczenia metodą Bayesowską dla KNN

In [ ]:
knn = KNeighborsClassifier()
opt_knn_christine = BayesSearchCV(knn, param_grid, n_iter=50, random_state=42)
opt_knn_christine.fit(christine_X, christine_y)
ob_knn_score_christine = opt_knn_christine.cv_results_

In [ ]:
knn = KNeighborsClassifier()
opt_knn_click = BayesSearchCV(knn, param_grid, n_iter=50, random_state=42)
opt_knn_click.fit(click_X, click_y)
ob_knn_score_click = opt_knn_click.cv_results_

In [ ]:
knn = KNeighborsClassifier()
opt_knn_santa_nder = BayesSearchCV(knn, param_grid, n_iter=50, random_state=42)
opt_knn_santa_nder.fit(santa_nder_X, santa_nder_y)

In [ ]:
knn = KNeighborsClassifier()
opt_knn_kddc = BayesSearchCV(knn, param_grid, n_iter=50, random_state=42)
opt_knn_kddc.fit(kddc_X_transform, kddc_y)
ob_knn_score_kddc = opt_knn_kddc.cv_results_

In [ ]:
# Zapisywanie wyników
np.save('op_knn_score_click.npy', score_knn_click)
np.save('op_knn_score_christine.npy', score_knn_christine)
np.save('op_knn_score_santa_nder.npy', score_knn_santa_nder)
np.save('op_knn_score_kddc.npy', score_knn_kddc)

### Część III - CatBoost

In [ ]:
# Instalacja pakietu catboost
# !pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.5 MB/s eta 0:00:00


In [ ]:
# Wczytanie pakietu i stworzenie modelu BatBoost
from catboost import CatBoostClassifier

In [ ]:
# Określenie parametrów i stworzenie siatki
param_grid_catboost = {
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.5],
        'bagging_temperature': [0.1, 0.5, 1, 1.5],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'colsample_bylevel': [0.1, 0.5, 1.0]
}

Obliczenia dla Random Search w przypadku CatBoost

In [ ]:
# from sklearn.model_selection import GridSearchCV
# grid_search = GridSearchCV(estimator = catboost_model, param_grid = param_grid_catboost, cv = 5, scoring = 'accuracy')
cb_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid_catboost, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
catboost_model = CatBoostClassifier()
cb_random.fit(click_X, click_y)
best_model = cb_random.best_estimator_
score_catboost_click = cb_random.cv_results_
np.save('score_catboost_click.npy', score_catboost_click)

In [ ]:
catboost_model = CatBoostClassifier()
cb_random.fit(santa_nder_X, santa_nder_y)
best_model = cb_random.best_estimator_
score_catboost_santa_nder = cb_random.cv_results_
np.save('score_catboost_santa_nder.npy', score_catboost_santa_nder)

In [ ]:
catboost_model = CatBoostClassifier()
cb_random.fit(kddc_X_transform, kddc_y)
best_model = cb_random.best_estimator_
score_catboost_kddc = cb_random.cv_results_
np.save('score_catboost_kddc.npy', score_catboost_kddc)

In [ ]:
catboost_model = CatBoostClassifier(cat_features=list(categorical_columns_christine))
cb_random = GridSearchCV(estimator=catboost_model, param_grid=param_grid_catboost, cv=5, scoring='accuracy')
cb_random.fit(christine_X, christine_y)
best_model = cb_random.best_estimator_
score_catboost_christine = cb_random.cv_results_['mean_test_score']
np.save('score_catboost_christine.npy', score_catboost_christine)

Obliczenia metodą Bayesowską dla Catboost

In [ ]:
catboost_model = CatBoostClassifier()
opt = BayesSearchCV(catboost_model, param_grid_catboost, n_iter=50, random_state=42)
_ = opt.fit(click_X, click_y)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(click_X, click_y))
bs_cb_results_click = opt.cv_results_
np.save('bs_cb_results_click.npy', bs_cb_results_click)

In [ ]:
catboost_model = CatBoostClassifier()
opt = BayesSearchCV(catboost_model, param_grid_catboost, n_iter=50, random_state=42)
_ = opt.fit(santa_nder_X, santa_nder_y)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(santa_nder_X, santa_nder_y))
bs_cb_results_santa_nder = opt.cv_results_
np.save('bs_cb_results_santa_nder.npy', bs_cb_results_santa_nder)

In [ ]:
catboost_model = CatBoostClassifier()
opt = BayesSearchCV(catboost_model, param_grid_catboost, n_iter=50, random_state=42)
_ = opt.fit(kddc_X_transform, kddc_y)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(kddc_X_transform, kddc_y))
bs_cb_results_kddc = opt.cv_results_
np.save('bs_cb_results_kddc.npy', bs_cb_results_kddc)

In [ ]:
catboost_model = CatBoostClassifier(cat_features=list(categorical_columns_christine))
opt = BayesSearchCV(catboost_model, param_grid_catboost, n_iter=30, random_state=42)
_ = opt.fit(christine_X, christine_y)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(christine_X, christine_y))
bs_cb_results_christine = opt.cv_results_['mean_test_score']
np.save('bs_cb_results_christine.npy', bs_cb_results_christine)